## 04_CompareTwoJsonsAppendToListDifferencesSelectLastValue

Autor: Carlos Wong <br/>
email: carlosawongca@gmail.com <br/>
Fecha: 2023_03_20 <br/>
Objetivo: Comparación con respecto al dato json más reciente <br/>
Entrada: 'df_base_4.xlsx', 'main2.json' <br/>
Salida: main3.json <br/>
Propósito: Prueba diferentes componentes del proyecto Project Management_Data Analyst <br/>

In [1]:
# librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# extracción de json de referencia
with open(".\jsons\main2.json") as f:
    json_list1 = json.load(f)
json_list1

[{'Col0 - Ref1': {'value': 'Ref1', 'date_time': '2023-03-20 14:47'}},
 {'Col0 - Ref2': {'value': 'Ref2', 'date_time': '2023-03-20 14:47'}},
 {'Col1 - Ref1': {'value': 'Water', 'date_time': '2023-03-20 14:47'}},
 {'Col1 - Ref2': {'value': 'Fire', 'date_time': '2023-03-20 14:47'}},
 {'Col2 - Ref1': {'value': 'Jhon', 'date_time': '2023-03-20 14:47'}},
 {'Col2 - Ref2': {'value': 'Mike', 'date_time': '2023-03-20 14:47'}},
 {'Col2 - Ref2': {'value': 'Peter', 'date_time': '2023-03-20 15:19'}}]

## 01_Conversión de datos nuevos en nuevo json_list2

In [14]:
# extracción de dataset con dos modificaciones: 
# Col2 - Ref2: 'Rose' 
# Col1 - Ref2: 'Wind'
data = pd.read_excel(r'.\df_base_4.xlsx')
data

,Col0,Col1,Col2
0,Ref1,Water,Jhon
1,Ref2,Wind,Rose


In [6]:
# Función que convierte una entrada de tres elementos en un diccionario de un solo elemento que contiene el valor original, 
# la fecha y la hora.
def convert_to_json(col_name, row_name, value):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d %H:%M")
    json_dict = {f"{col_name} - {row_name}": {"value": value, "date_time": date_time}}
    return json_dict

In [7]:
# convierte cada celda del DataFrame df en un diccionario en formato JSON y lo agrega a la lista json_list.
json_list2 = []
for col_name in data.columns:
    for index, row in data.iterrows():
        row_name = row['Col0']
        value = row[col_name]
        json_dict = convert_to_json(col_name, row_name, value)
        json_list2.append(json_dict)
json_list2      

[{'Col0 - Ref1': {'value': 'Ref1', 'date_time': '2023-03-20 16:18'}},
 {'Col0 - Ref2': {'value': 'Ref2', 'date_time': '2023-03-20 16:18'}},
 {'Col1 - Ref1': {'value': 'Water', 'date_time': '2023-03-20 16:18'}},
 {'Col1 - Ref2': {'value': 'Wind', 'date_time': '2023-03-20 16:18'}},
 {'Col2 - Ref1': {'value': 'Jhon', 'date_time': '2023-03-20 16:18'}},
 {'Col2 - Ref2': {'value': 'Rose', 'date_time': '2023-03-20 16:18'}}]

## 02_Comparación json_list1 y json_list2

In [8]:
differences = []

for item in json_list2:
    for key, value in item.items():
        col_ref = key.split(' - ')[0] + ' - ' + value['value']
        # buscar solo en el valor, excluyendo la fecha
        ref_value = {'value': value['value']}
        if not any(d.get(key, {}).get('value') == value['value'] for d in json_list1):
            differences.append({key: value})           

In [9]:
differences

[{'Col1 - Ref2': {'value': 'Wind', 'date_time': '2023-03-20 16:18'}},
 {'Col2 - Ref2': {'value': 'Rose', 'date_time': '2023-03-20 16:18'}}]

## 03_Agrego las diferencias a la lista json original

In [10]:
json_list1.extend(differences)
json_list1

[{'Col0 - Ref1': {'value': 'Ref1', 'date_time': '2023-03-20 14:47'}},
 {'Col0 - Ref2': {'value': 'Ref2', 'date_time': '2023-03-20 14:47'}},
 {'Col1 - Ref1': {'value': 'Water', 'date_time': '2023-03-20 14:47'}},
 {'Col1 - Ref2': {'value': 'Fire', 'date_time': '2023-03-20 14:47'}},
 {'Col2 - Ref1': {'value': 'Jhon', 'date_time': '2023-03-20 14:47'}},
 {'Col2 - Ref2': {'value': 'Mike', 'date_time': '2023-03-20 14:47'}},
 {'Col2 - Ref2': {'value': 'Peter', 'date_time': '2023-03-20 15:19'}},
 {'Col1 - Ref2': {'value': 'Wind', 'date_time': '2023-03-20 16:18'}},
 {'Col2 - Ref2': {'value': 'Rose', 'date_time': '2023-03-20 16:18'}}]

## 04_Guardo nuevo fichero json

In [13]:
with open(f".\jsons\main_{pd.datetime.now().strftime('%Y%m%d_%H%M')}.json", "w") as f:
    json.dump(json_list1, f)